# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [4]:
import pandas as pd
import numpy as np
import io
from matplotlib import pyplot as plt

from google.colab import files
uploaded = files.upload()


Saving Game_Logs_Quarterback.csv to Game_Logs_Quarterback (1).csv


## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [54]:
df = pd.read_csv(io.BytesIO(uploaded['Game_Logs_Quarterback.csv']))

brees = df[(df['Player Id'] == 'drewbrees/2504775') & (df['Season'] == 'Regular Season')]


#brees.Week.value_counts()
#brees.Year.value_counts()

#brees.head()

#brees.columns

efficiency = brees[['Home or Away', 'Completion Percentage', 'Passing Yards Per Attempt', 'Passer Rating']].reset_index()
efficiency.columns

'''
production = brees[['Home or Away', 'Passing Yards', 'TD Passes']]
mistakes = brees[['Home or Away', 'Sacks', 'Ints', 'Fumbles']]

efficiency = efficiency.groupby('Home or Away').mean()
'''


"\nproduction = brees[['Home or Away', 'Passing Yards', 'TD Passes']]\nmistakes = brees[['Home or Away', 'Sacks', 'Ints', 'Fumbles']]\n\nefficiency = efficiency.groupby('Home or Away').mean()\n"

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.